# Probabilidade de infecção por agente infeccioso
Como vimos durante a análise exploratória, cerca de 54% dos casos de SRAGs no 
Brasil não possuem uma conclusão sobre o agente infeccioso. Por isso, vamos
treinar um modelo de classificação para tentar prever qual o agente infeccioso
de um caso de SRAG.

In [1]:
import re
import pandas as pd
from ydata_profiling import ProfileReport

In [2]:
# Importando os dados
data = pd.read_csv("../data/raw/INFLUD23-16-10-2023.csv", sep=";")

/tmp/ipykernel_12636/60331149.py:2: DtypeWarning: Columns (15,53,63,91,93,143) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/raw/INFLUD23-16-10-2023.csv", sep=";")


Nem todas as colunas disponíveis no dataset serão utilizadas para a 
classificação. Após avaliação inicial, as seguintes colunas foram
selecionadas para a classificação: DT_SIN_PRI, SEM_PRI, SG_UF_NOT, ESTRANG, 
NU_IDADE_N, POV_CT, SG_UF, CS_ZONA, OUT_ANIM, FEBRE, TOSSE, GARGANTA, DISPNEIA, 
DESC_RESP, SATURACAO, DIARREIA, VOMITO, DOR_ABD, FADIGA, PERD_OLF, PERD_PALA, 
OUTRO_SIN, OUTRO_DES, VACINA, DOSE_2REF, CLASSI_FIN, CLASSI_OUT.

Primeiro, temos a data dos primeiros sintomas (DT_SIN_PRI) e a semana de
primeiros sintomas (SEM_PRI). Tais colunas foram selecionadas por certos agentes
infecciosos possuirem sazonalidade, como é o caso do vírus influenza. As 
variáveis de idade (NU_IDADE_N), estado (SG_UF), estrangeiro (ESTRANG) e zona 
(CS_ZONA) foram selecionadas por serem variáveis demográficas que podem 
influenciar na probabilidade de infecção. Outra que pode influenciar é se o 
paciente teve contato com certos animais (OUT_ANIM). As variáveis de sintomas 
(FEBRE, TOSSE, GARGANTA, DISPNEIA, DESC_RESP, SATURACAO, DIARREIA, VOMITO, 
DOR_ABD, FADIGA, PERD_OLF, PERD_PALA, OUTRO_SIN, OUTRO_DES) foram selecionadas 
por serem sintomas comuns de SRAGs. Por último, as variáveis de vacinação 
(VACINA, DOSE_2REF) foram selecionadas por serem variáveis que revelam se o 
paciente foi vacinado contra certos agentes infecciosos. 

As variáveis de classificação (CLASSI_FIN, CLASSI_OUT) foram selecionadas por
serem as variáveis que queremos prever. A variável CLASSI_FIN é a classificação
final do caso, enquanto CLASSI_OUT é um complemento. Ambas serão unidas em uma
única variável, chamada CLASSI, que será a variável alvo do modelo.



In [3]:
# Criando lista com as colunas selecionadas
colunas_selecionadas = ["DT_SIN_PRI", "SEM_PRI", "ESTRANG", 
"NU_IDADE_N", "SG_UF", "CS_ZONA", "OUT_ANIM", "FEBRE", "TOSSE", 
"GARGANTA", "DISPNEIA", "DESC_RESP", "SATURACAO", "DIARREIA", "VOMITO", 
"DOR_ABD", "FADIGA", "PERD_OLFT", "PERD_PALA", "OUTRO_SIN", "OUTRO_DES", 
"VACINA", "VACINA_COV", "DOSE_2REF", "CLASSI_FIN", "CLASSI_OUT"]

In [4]:
# Buscando as colunas selecionadas
data = data[colunas_selecionadas]

In [5]:
# Verificando os dados
data.head()

,DT_SIN_PRI,SEM_PRI,ESTRANG,NU_IDADE_N,SG_UF,CS_ZONA,OUT_ANIM,FEBRE,TOSSE,GARGANTA,...,FADIGA,PERD_OLFT,PERD_PALA,OUTRO_SIN,OUTRO_DES,VACINA,VACINA_COV,DOSE_2REF,CLASSI_FIN,CLASSI_OUT
0,17/01/2023,3,2.0,75,MG,1.0,NaN,1.0,1.0,2.0,...,1.0,2.0,2.0,2.0,NaN,1.0,1.0,11/04/2022,4.0,NaN
1,01/01/2023,1,2.0,67,RJ,1.0,NaN,1.0,9.0,9.0,...,9.0,9.0,9.0,9.0,NaN,9.0,1.0,10/05/2022,4.0,NaN
2,05/01/2023,1,NaN,72,SP,1.0,NaN,2.0,1.0,2.0,...,2.0,2.0,2.0,2.0,NaN,NaN,1.0,19/04/2022,5.0,NaN
3,18/01/2023,3,2.0,46,SP,1.0,NaN,1.0,2.0,NaN,...,2.0,2.0,2.0,1.0,"FRAQUEZA,MAL ESTAR,MIALGIA",2.0,1.0,NaN,4.0,NaN
4,03/02/2023,5,2.0,71,SP,NaN,NaN,NaN,1.0,2.0,...,2.0,2.0,2.0,NaN,NaN,1.0,1.0,25/05/2022,4.0,NaN


In [6]:
# Verificando as dimensões do dataset
data.shape

(226763, 26)

Vamos usar o Pandas Profilling para gerar um report sobre os dados e assim decidir qual caminho seguir.

In [7]:
# Gerando o report
#profile = ProfileReport(data, title="Pandas Profiling Report")
#profile.to_file("../images/reports/INFLUD23-16-10-2023.html")

Com base no relatório, as seguintes modificações serão feitas:
- Selecionar apenas os registros com o agente infeccioso definido;
- Alterar o tipo das colunas DT_SIN_PRI e DOSE_2REF para datetime;
- Deletar os registros negativos da coluna NU_IDADE_N (Não existe idade negativa);
- Por ter 99.7% de missing, a coluna OUT_ANIM será deletada;
- Expandir a coluna OUTRO_DES para gerar novas colunas com sintomas;
- Unir a coluna CLASSI_OUT com a coluna CLASSI_FIN;
- Alterar todos os valores numéricos das colunas para o seu real valor;

Insights gerais:
- 28% dos dados estão ausentes.
- 0.4% dos dados são duplicados.
- Temos um problema de classificação com multiplos rótulos.

## Aplicando as correções no dataset

### Selecionando apenas os registros com o agente infeccioso definido

In [8]:
# Selecionando apenas os casos identificados
data = data.query("CLASSI_FIN != 4.0")

# Dropando os registros com valores nulos no target
data = data.dropna(subset=["CLASSI_FIN"])

### Alterando o tipo das colunas DT_SIN_PRI e DOSE_2REF para datetime

In [9]:
# Alterando o tipo dos dados
data["DT_SIN_PRI"] = pd.to_datetime(data["DT_SIN_PRI"], format="%d/%m/%Y")
data["DOSE_2REF"] = pd.to_datetime(data["DOSE_2REF"], format="%d/%m/%Y")

### Deletar os registros negativos da coluna NU_IDADE_N (Não existe idade negativa)

In [10]:
# Buscando apenas os registros com idade maior ou igual a zero
data = data.query("NU_IDADE_N >= 0")

### Por ter 99.7% de missing, a coluna OUT_ANIM será deletada

In [11]:
# Dropando a coluna
data = data.drop("OUT_ANIM", axis = 1)

### Expandir a coluna OUTRO_DES para gerar novas colunas com sintomas

In [12]:
# Criando uma bag com os sintomas
lista_sintomas_inicial = []

# Criando um loop para unir todos os sintomas
for lista_sintomas in data.OUTRO_DES.str.split(",").dropna():
    lista_sintomas_inicial.extend(lista_sintomas)
    
# Removendo os espaços em branco
lista_sintomas_inicial = [sintoma.strip() for sintoma in lista_sintomas_inicial]

# Removendo palavras duplicadas
lista_sintomas = list(set(lista_sintomas_inicial))

# Removendo caracteres especiais
lista_sintomas = [re.sub("[^A-Za-z0-9]+", " ", sintoma) for sintoma in lista_sintomas]

# Separando as palavras que possuem a letra 'E'
lista_sintomas = [sintoma.split(" E ") for sintoma in lista_sintomas]

# Criando uma lista com todos os sintomas
lista_sintomas_final = []

# Criando um loop para unir todos os sintomas
for lista_sintomas in lista_sintomas:
    lista_sintomas_final.extend(lista_sintomas)
    
# Removendo palavras duplicadas
lista_sintomas_final = list(set(lista_sintomas_final))

In [13]:
## Criando um vetor com os sintomas
#for sintoma in lista_sintomas_final:
#    data[sintoma] = data["OUTRO_DES"].str.contains(sintoma, na=False)

# Dropando a coluna por enquanto
data = data.drop("OUTRO_DES", axis = 1)

### Unir a coluna CLASSI_OUT com a coluna CLASSI_FIN

In [14]:
# Alterando os valores numéricos para os nomes das categorias
dict_novas_cat = {1.0:"Influenza",
                  2.0:"Outro vírus",
                  3.0:3.0,
                  5.0:"Covid-19"}

# Aplicando a mudança
data["CLASSI_FIN"] = data["CLASSI_FIN"].map(dict_novas_cat)

# Buscando as classes complementares
data.loc[data["CLASSI_FIN"] == 3, "CLASSI_FIN"] = data.loc[data["CLASSI_FIN"] == 3, "CLASSI_OUT"]

# Dropando a coluna CLASSI_OUT
data = data.drop("CLASSI_OUT", axis = 1)

### Alterar todos os valores numéricos das colunas para o seu real valor;

In [15]:
# Criando as funções
func_estrangeiro = lambda num: "Sim" if num == 1 else "Não"
func_zona = lambda num: "Urbana" if num == 1 else ("Rural" if num == 2 else ("Periurbana" if num == 3 else "Ignorado"))
func_sintoma_and_vacina = lambda num: "Sim" if num == 1 else ("Não" if num == 2 else "Ignorado")

# Aplicando as alterações
data["ESTRANG"] = data["ESTRANG"].apply(func_estrangeiro)
data["CS_ZONA"] = data["CS_ZONA"].apply(func_zona)

# Criando uma lista com os sintomas
sintomas = ["FEBRE", "TOSSE", "GARGANTA", "DISPNEIA", "DESC_RESP", "SATURACAO", 
            "DIARREIA", "VOMITO", "DOR_ABD", "FADIGA", "PERD_OLFT", 
            "PERD_PALA", "OUTRO_SIN"]

# Aplicando as alterações
for sintoma in sintomas:
    data[sintoma] = data[sintoma].apply(func_sintoma_and_vacina)
    
# Criando uma lista com as vacinas
vacinas = ["VACINA", "VACINA_COV"]
for vacina in vacinas:
    data[vacina] = data[vacina].apply(func_sintoma_and_vacina)


## Engenharia de Features

Aqui, irei criar as seguintes features:
- Diferença entre a data dos primeiros sintomas e a data de reforço para covid;
- Verificar de qual região é o estado do paciente;
- Se ele apresenta multiplos sintomas;


In [17]:
# Calculando a diferença de dias entre a data de vacinação da covid e a data de sintomas
data['DIFERENCA_DIAS'] = (data['DT_SIN_PRI'] - data['DOSE_2REF']).dt.days

In [18]:
# Criando uma lista com as UF de cada região
regiao_sul = ["PR", "SC", "RS"]
regiao_sudeste = ["SP", "RJ", "MG", "ES"]
regiao_centro_oeste = ["MS", "MT", "GO", "DF"]
regiao_norte = ["RO", "AC", "AM", "RR", "PA", "AP", "TO"]

# Criando uma função para identificar a região
func_regiao = lambda uf: "Sul" if uf in regiao_sul else("Sudeste" if uf in regiao_sudeste else("Centro-Oeste" if uf in regiao_centro_oeste else("Norte" if uf in regiao_norte else "Nordeste")))
data["REGIAO"] = data["SG_UF"].apply(func_regiao)

In [20]:
# Contando a quantidade de sintomas que o paciente apresentou
data["MAIS_DE_UM_SINTOMA"] = data[sintomas].apply(lambda x: x.str.contains("Sim").sum(), axis=1)

## Realizando experimentos